<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Introduction

In this lab, we will learn how to convert addresses into their equivalent latitude and longitude values. Also, we will use the Foursquare API to explore neighborhoods in Toronto. We will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the _k_-means clustering algorithm to complete this task. Finally, we will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in Toronto</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>
  
    </font>
    </div>

Lets first import the required libraries....

In [6]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np

from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset


In [7]:
#!pip install beautifulsoup4
#!pip install lxml
#!pip install html5lib
!pip install geocoder

**For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. we  will use BeautifulSoup to scrape the Wikipedia, page wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format.**

In [8]:
import html5lib
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'#Wikipedia url of the list of neighborhood in toronto
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib") #Creating a BeautifulSoup Object
table_contents=[]
table=soup.find('table')
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append({'Postal Code':cell['PostalCode'],'Borough':cell['Borough'],'Neighborhood':cell['Neighborhood']})
# print(table_contents)
neighborhoods=pd.DataFrame(table_contents)
neighborhoods['Borough']=neighborhoods['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [9]:
neighborhoods.shape

(103, 3)

 Running a while loop for each postal code to get neighborhoods coordinates.

In [10]:
import geocoder

geolocator = Nominatim(user_agent="toronto_explorer")

lat_col = []
lon_col = []

for borough in neighborhoods['Borough']:
        
    location = geolocator.geocode(borough + ', Toronto')
    latitude = location.latitude
    longitude = location.longitude   
    
    lat_col.append(latitude)
    lon_col.append(longitude)

# Insert the Latitude and Longitude columns to the dataframe df
neighborhoods['Latitude']  = lat_col
neighborhoods['Longitude'] = lon_col

In [11]:
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754326,-79.449117
1,M4A,North York,Victoria Village,43.754326,-79.449117
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654174,-79.380812
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.754326,-79.449117
4,M7A,Queen's Park,Ontario Provincial Government,43.659659,-79.390340


Let's slice the original dataframe and create a new dataframe of the Toronto data.

In [12]:
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto',regex=False)].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654174,-79.380812
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.654174,-79.380812
2,M5C,Downtown Toronto,St. James Town,43.654174,-79.380812
3,M4E,East Toronto,The Beaches,43.626243,-79.396962
4,M5E,Downtown Toronto,Berczy Park,43.654174,-79.380812


Getting the geographical coordinates of Toronto.


In [13]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Foursquare Credentials and Version


In [15]:
CLIENT_ID = 'XIZOSZ0MA5TQ4AAAZM3SJPUECLQ0JG3BY51WR0UL55Z4N0SP' # Foursquare ID
CLIENT_SECRET = 'JIKFNLRBQFHPM35G3YA5YUI02GU0TUXPH4MVJ4ISVTONAAPM' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XIZOSZ0MA5TQ4AAAZM3SJPUECLQ0JG3BY51WR0UL55Z4N0SP
CLIENT_SECRET:JIKFNLRBQFHPM35G3YA5YUI02GU0TUXPH4MVJ4ISVTONAAPM


#### Let's explore the first neighborhood in our dataframe.


In [16]:
toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [17]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6541737, -79.38081162653639.


#### Now, let's get the top 100 venues that are  within a radius of 500 meters.


In [18]:
# let's create the GET request URL.

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude, 
    neighborhood_longitude,
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XIZOSZ0MA5TQ4AAAZM3SJPUECLQ0JG3BY51WR0UL55Z4N0SP&client_secret=JIKFNLRBQFHPM35G3YA5YUI02GU0TUXPH4MVJ4ISVTONAAPM&v=20180605&ll=43.6541737,-79.38081162653639&radius=500&limit=100'

GET request and  resutls


In [19]:
results = requests.get(url).json()

Let's define a function that extract the category of a venue.. We will name that function get_category_type.

In [20]:
# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now it is time to clean the json and structure it into a _pandas_ dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-21-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Elgin And Winter Garden Theatres,Theater,43.653394,-79.378507
1,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
2,LUSH,Cosmetics Shop,43.653557,-79.380400
3,Ed Mirvish Theatre,Theater,43.655102,-79.379768
4,Yonge-Dundas Square,Plaza,43.656054,-79.380495


## 2. Explore Neighborhoods in Toronto


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [22]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Running the above function on each neighborhood and create a new dataframe called _toronto_venues_.


In [23]:
names = toronto_data['Neighborhood']
latitudes =  toronto_data['Latitude']
longitudes= toronto_data['Longitude']

toronto_venues = getNearbyVenues(names, latitudes, longitudes, radius=500)
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654174,-79.380812,Elgin And Winter Garden Theatres,43.653394,-79.378507,Theater
1,"Regent Park, Harbourfront",43.654174,-79.380812,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store
2,"Regent Park, Harbourfront",43.654174,-79.380812,LUSH,43.653557,-79.380400,Cosmetics Shop
3,"Regent Park, Harbourfront",43.654174,-79.380812,Ed Mirvish Theatre,43.655102,-79.379768,Theater
4,"Regent Park, Harbourfront",43.654174,-79.380812,Yonge-Dundas Square,43.656054,-79.380495,Plaza
...,...,...,...,...,...,...,...
2909,Church and Wellesley,43.654174,-79.380812,Subway,43.656953,-79.377296,Sandwich Place
2910,Church and Wellesley,43.654174,-79.380812,The Ram in the Rye,43.657946,-79.377836,Bar
2911,Church and Wellesley,43.654174,-79.380812,Good View Chinese Restaurant,43.656791,-79.376139,Chinese Restaurant
2912,Church and Wellesley,43.654174,-79.380812,Alibasha - Shisha bar,43.656775,-79.375761,Bar


## 3. Analyze Each Neighborhood


In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Vietnamese Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Bookstore,Breakfast Spot,...,Smoke Shop,Smoothie Shop,Speakeasy,Steakhouse,Sushi Restaurant,Tanning Salon,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [25]:
toronto_onehot.shape

(2914, 81)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Vietnamese Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Bookstore,...,Smoke Shop,Smoothie Shop,Speakeasy,Steakhouse,Sushi Restaurant,Tanning Salon,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store
0,Berczy Park,0.011494,0.011494,0.011494,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,...,0.000000,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,0.022989,0.000000,0.011494
1,"Brockton, Parkdale Village, Exhibition Place",0.011236,0.022472,0.000000,0.011236,0.011236,0.011236,0.011236,0.011236,0.011236,...,0.011236,0.011236,0.011236,0.011236,0.022472,0.011236,0.022472,0.022472,0.011236,0.011236
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.011494,0.011494,0.011494,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,...,0.000000,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,0.022989,0.000000,0.011494
3,Central Bay Street,0.011494,0.011494,0.011494,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,...,0.000000,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,0.022989,0.000000,0.011494
4,Christie,0.011494,0.011494,0.011494,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,...,0.000000,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,0.022989,0.000000,0.011494
5,Church and Wellesley,0.011494,0.011494,0.011494,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,...,0.000000,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,0.022989,0.000000,0.011494
6,"Commerce Court, Victoria Hotel",0.011494,0.011494,0.011494,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,...,0.000000,0.011494,0.000000,0.000000,0.011494,0.022989,0.011494,0.022989,0.000000,0.011494
7,Davisville,0.011236,0.022472,0.000000,0.011236,0.011236,0.011236,0.011236,0.011236,0.011236,...,0.011236,0.011236,0.011236,0.011236,0.022472,0.011236,0.022472,0.022472,0.011236,0.011236
8,Davisville North,0.011236,0.022472,0.000000,0.011236,0.011236,0.011236,0.011236,0.011236,0.011236,...,0.011236,0.011236,0.011236,0.011236,0.022472,0.011236,0.022472,0.022472,0.011236,0.011236
9,"Dufferin, Dovercourt Village",0.011236,0.022472,0.000000,0.011236,0.011236,0.011236,0.011236,0.011236,0.011236,...,0.011236,0.011236,0.011236,0.011236,0.022472,0.011236,0.022472,0.022472,0.011236,0.011236


#### Now we print each neighborhood along with the top 5 most common venues


In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
   # print('\n')

#### Let's put that into a _pandas_ dataframe


In [28]:
#First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Clothing Store,Restaurant,Diner,Hotel,Seafood Restaurant,Café,Gastropub,Bar,Middle Eastern Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Clothing Store,Hotel,Restaurant,Café,Sushi Restaurant,Salad Place,Theater,Thai Restaurant,Pizza Place
2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Clothing Store,Restaurant,Diner,Hotel,Seafood Restaurant,Café,Gastropub,Bar,Middle Eastern Restaurant
3,Central Bay Street,Coffee Shop,Clothing Store,Restaurant,Diner,Hotel,Seafood Restaurant,Café,Gastropub,Bar,Middle Eastern Restaurant
4,Christie,Coffee Shop,Clothing Store,Restaurant,Diner,Hotel,Seafood Restaurant,Café,Gastropub,Bar,Middle Eastern Restaurant


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 4 clusters.


In [30]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 0, 3, 3, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654174,-79.380812,0,Coffee Shop,Clothing Store,Restaurant,Diner,Hotel,Seafood Restaurant,Café,Gastropub,Bar,Middle Eastern Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.654174,-79.380812,0,Coffee Shop,Clothing Store,Restaurant,Diner,Hotel,Seafood Restaurant,Café,Gastropub,Bar,Middle Eastern Restaurant
2,M5C,Downtown Toronto,St. James Town,43.654174,-79.380812,0,Coffee Shop,Clothing Store,Restaurant,Diner,Hotel,Seafood Restaurant,Café,Gastropub,Bar,Middle Eastern Restaurant
3,M4E,East Toronto,The Beaches,43.626243,-79.396962,1,Performing Arts Venue,Hotel,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Diner,Electronics Store
4,M5E,Downtown Toronto,Berczy Park,43.654174,-79.380812,0,Coffee Shop,Clothing Store,Restaurant,Diner,Hotel,Seafood Restaurant,Café,Gastropub,Bar,Middle Eastern Restaurant


Finally, let's visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Thanks for completing this lab...!**